In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
wikipediaPage = requests.get("https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich")

In [ ]:
# import it in a soup object
soup = BeautifulSoup(wikipediaPage.content, 'html5lib')

# get the table with the needed info
table = soup.find_all('table')[1]

# extract all <tr> from the table
trs = table.find_all('tr')

In [ ]:
# function to parse the <tr> and return an entry for the dataframe : return Borough, Neighborhood
def parseTableRow(tr, index):
    borough = 'District ' + str(index)
    
    neighborhood = ''

    td = tr.find_next('td')
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    
    for string in td.stripped_strings:
        if (neighborhood != ''):
            neighborhood = neighborhood + ', '
        neighborhood = neighborhood + string
    
    return borough, neighborhood

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

In [ ]:
# fill the dataframe
index = 1
for tr in trs:
    if 'District ' in str(tr):
        data = parseTableRow(tr, index)
        neighborhoods = neighborhoods.append({'Borough': data[0], 'Neighborhood': data[1]}, ignore_index=True)
        index = index + 1

In [ ]:
neighborhoods

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
baseUrl = 'https://maps.googleapis.com/maps/api/geocode/json?'

In [ ]:
def getLocation(district):
    """ 
    Search for information for place Zürich<district>
    return the found place latitude and longitude (return latitude, longitude)
    """
    url = (baseUrl+'&address={}').format('Zürich'+district)
    response = requests.get(url).json() # get response
    
    success = False
    while not success:
        try:
            geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
            success = True
        except: 
            pass
    
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    return latitude, longitude

In [ ]:
nh = neighborhoods.copy()

latitudeCln = []
longitudeCln = []
for index, row in nh.iterrows():
    print(row[0])
    lat, long = getLocation(row[0])
    latitudeCln.append(lat)
    longitudeCln.append(long)

nh['Latitude'] = latitudeCln
nh['Longitude'] = longitudeCln

nh.shape

In [ ]:
nh

In [ ]:
address = 'Zürich, Switzerland'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zürich are {}, {}.'.format(latitude, longitude))

In [ ]:
# make a copy of the dataframe to get it simply back if needed
nhz = nh.copy()

In [ ]:
# create map of Zürich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(nhz['Latitude'], nhz['Longitude'], nhz['Borough'], nhz['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# function to repeat the exploring process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        if (categoryIds != ''):
            url = url + '&categoryId={}'
            url.format(categoryIds)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

In [ ]:
zh_venues = getNearbyVenues(names=nhz['Neighborhood'], latitudes=nhz['Latitude'], longitudes=nhz['Longitude'])

In [ ]:
print(zh_venues.shape)
zh_venues.head()

In [ ]:
zh_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(zh_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
zh_onehot = pd.get_dummies(zh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zh_onehot['Neighborhood'] = zh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zh_onehot.columns[-1]] + list(zh_onehot.columns[:-1])
zh_onehot = zh_onehot[fixed_columns]

zh_onehot.head()

In [ ]:
zh_grouped = zh_onehot.groupby('Neighborhood').sum().reset_index()
zh_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = zh_grouped['Neighborhood']

for ind in np.arange(zh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(zh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted